# Wstęp
Celem projektu było zastosowanie równania ciepła do symulacji ogrzewania pomieszczeń. W projekcie zostały poruszone dwa zasadnicze problemy badawcze:
1. *Czy grzejnik powinien znajdować się pod oknem?*
2. *Czy warto wyłączać grzejniki przed wyjściem z domu?*

Symulacje dla pierwszego problemu zostały przeprowadzone na uproszczonym modelu składającego się z tylko jednego pokoju, podczas gdy symulacje drugiego zagadnienia zostały przeprowadzone na modelu bazującym na prawdziwym mieszkaniu.
Każdemu z tych zagadnień został poświęcony osobny rozdział w dalszej części raportu.

# Opis projektu

## Rozważania teoretyczne

Klasyczne równanie ciepła ma postać
$$u(x,t) = \alpha \Delta u(x,t),$$
gdzie $\alpha$ to *współczynnik przewodnictwa*, a $\Delta$ to laplasjan $\Delta = \frac{\partial^2}{\partial {x_1}^2} + \frac{\partial^2}{\partial {x_2}^2}$. Zakładamy dla uproszczenia, że mieszkanie jest dwuwymiarowe, oraz że $u(x,t)$ oznacza temperaturę w punkcie $x$ w czasie $t$, gdzie $x$ jest wektorem dwuwymiarowym.

Mieszkanie $\Omega \subseteq \mathbb{R}^2$, które interpretujemy jako pewien prostokąt na płaszczyźnie, dzielimy na obszary: pokoje $\mathcal{P}$, grzejniki $\mathcal{G}$, okna $\mathcal{O}$, ściany $\mathcal{S}$ oraz drzwi $\mathcal{D}$. Obszary te nie muszą być rozłączne. 

Do powyższego równania ciepła wprowadzamy modyfikację:
$$u(x,t) = \alpha \Delta u(x,t) + f(x,t) \text{   } (\clubsuit),$$
gdzie $f$ jest funkcją symulującą działanie grzejnika w punkcie $x$ i o czasie $t$. Równanie to będzie stosowane do symulacji rozprowadzania się temperatury w pokojach. Dla większości obszaru mieszkania $f \equiv 0$.
Rozchodzenie się ciepła na poszczególnych obszarach w czasie opisujemy następującymi rówaniami:
$$
\begin{cases}
\\ u(x,t) = \alpha \Delta u(x,t) + f(x,t), x\in P \in \mathcal{P};
\\ u(x,t) = T_{out} (t), x \in O \in \mathcal{O};
\\ \nabla u \cdot n = 0, x \in S \in \mathcal{S};
\\ u(x,t) = \frac{1}{|D|} \int_{D} u(s,t) ds, x\in D \in \mathcal{D};
\\ u(x,0) = u_{0}(x), x\in \Omega,
\end{cases}
$$

gdzie $T_{out} (t)$ to temperatura na dworze w czasie $t$ oraz $n$ to wektor normalny do ściany.

Równania te mówią nam, że:
- w pokojach temperatura rozchodzi się zgodnie z równaniem $(\clubsuit)$;
- okna zawsze przyjmują temperaturę, która panuje na zewnątrz;
- ściany są idealnymi izolatorami;
- temperatura równo rozprzestrzenia się w drzwiach.

### Opis metody numerycznej

Metodą numeryczną, którą wykorzystamy do rozwiązania zagadnienia jest metoda różnic skończonych. Oto wzór, którego użyjemy do symulacji dyfuzji ciepła:
$$
u(x,y,t+h_t) = u(x,y,t) + \frac{\alpha h_t}{h_x^2} \Big( u(x+h_x, y, t) + u(x-h_x, y, t) + u(x, y-h_x, t) + u(x,y+h_x, t) - 4 u(x,y,t) \Big) + h_t f(x,t) \text{      } (\spadesuit)
$$

Więcej na temat tej metody oraz wyprowadzenie powyższego wzoru można znaleźć w pracy pod poniższym linkiem: [LINK](https://www.researchgate.net/publication/265202152_Finite-Difference_Approximations_to_the_Heat_Equation). Wypisany powyżej wzór ma numer (16).

## Rozważania implementacyjne

Całość projektu została napisana w języku Python. Wykorzystany został paradygmat programowania obiektowego. Do implementacji numerycznej użyto funkcjonalności z biblioteki *NumPy*, z kolei do prezentacji wyników użyto wykresów wygenerowanych przez bibliotekę *PyPlotLib*. Mieszkanie oraz każdy rodzaj obszaru z poprzedniego podpunktu jest pewną klasą zawierającą informacje potrzebne do symulacji; w szczególności klasa `Apartment` zawiera listy obiektów przynależących do klas reprezentujących pokoje, grzejniki *et cetera*. Temperatura w symulacji jest wyrażona w stopniach Kelwina. Ściany nie są osobną klasą; zamiast tego są całkowicie pomijane w symulacji.

### Dyskretyzacja mieszkania

Mieszkanie użyte w drugim eskperymencie ma w rzeczywistości około $10$ metrów długości oraz $5$ metrów szerokości. Oznacza to, że blabla

### Dokładniejszy opis klas

#### Mieszkanie
`Apartment` to najbardziej rozbudowana klasa w projekcie. Obiekty przynależące do tej klasy zawierają następujące parametry:
1. `base_temp`: początkowa temperatura w mieszkaniu, zakładamy, że jest stała;
2. `coeff`: współczynnik $\alpha$ używany w równaniu przewodnictwa cieplnego;
3. `temp_outside`: funkcja symulujące zmieniającą się temperaturę na zewnątrz w ciągu doby;
4. `size`: wymiary macierzy reprezentującej mieszkanie;
5. `hx`: współczynnik używany w równaniu xxx;
6. `T`: czas symulacji wyrażony w godzinach;
7. `ht`: długość odstępu czasu, na które dzielimy `T`;
8. `n_rooms`: liczba pokoi;
9. `n_radiators`: liczba grzejników;
10. `n_windows`: liczba okien;
11. `n_doors`: liczba drzwi.

Dodatkowo funkcja `__init()` inicjalizuje następujące parametry:
1. `n_timeslips`: liczba iteracji algorytmu z xxx;
2. `rooms`: lista pokoi, początkowo pusta;
3. `radiators`: lista grzejników, początkowo pusta;
4. `windows`: lista okien, początkowo pusta;
5. `doors`: lista drzwi, początkowo pusta;
6. `Matrix`: trójwymiarowa macierz z biblioteki *NumPy*, na której wykonywana jest symulacja cieplna mieszkania. Pierwsza wszpółrzędna to numer iteracji algorytmu xxx, tzn. czas, podczas gdy kolejne dwie współrzędne opisują mieszkanie na płaszczyźnie.

Klasa `Apartment` zawiera następujące metody:
1. `add_room()`, `add_radiator()`, `add_window()`, `add_door()`, które służą do dodawania poszczególnych rodzajów obiektów do mieszkania;
2. `simulate()`, która pozwala na wykonanie symulacji.

#### Pokoje
Klasa `Room` składa się z następujących parametrów:
1. `x_coords`: opisują odcięte lewego dolnego oraz prawego górnego kąta pokoju;
2. `y_coords`: opisują rzędne lewego dolnego oraz prawego górnego kąta pokoju.

Klasa ta opisuje rozmieszczenie pokoi w mieszkaniu na podstawie dwóch par współrzędnych w rogach pokoju. Metoda `Apartment.simulate()` używa wzoru $(\spadesuit)$ do wyliczenia rozmieszczenia temperatury w kolejnych iteracjach.

#### Grzejniki
Klasa `Radiator` składa się z następujących parametrów:
1. `x_coords`: opisują odcięte lewego dolnego oraz prawego górnego kąta grzejnika;
2. `y_coords`: opisują rzędne lewego dolnego oraz prawego górnego kąta grzejnika;
3. `power`: trójargumentowa funkcja opisująca moc (ciepło oddawane) grzejnika w punkcie `(x, y)` i w czasie `t`.

Grzejnik po prostu dodaje temperaturę w każdym swoim punkcie zgodnie z funkcją `power`.

#### Okna
Klasa `Window` składa się z następujących parametrów:
1. `x_coords`: opisują odcięte lewego dolnego oraz prawego górnego kąta okna;
2. `y_coords`: opisują rzędne lewego dolnego oraz prawego górnego kąta okna.

W kolejnych iteracjach temperatura okien jest ustawiana na temperaturę na zewnątrz zgodnie z funkcją `self.temp_outside`.

#### Drzwi
Klasa `Door` składa się z następujących parametrów:
1. `x_coords`: opisują odcięte lewego dolnego oraz prawego górnego kąta drzwi;
2. `y_coords`: opisują rzędne lewego dolnego oraz prawego górnego kąta drzwi.
3. `direction`: wartość opisująca skierowanie drzwi (poziome lub pionowe).

Temperatura drzwi jest uśrednioną temperaturą z góry i z dołu dla poziomego skierowania drzwi oraz z lewa i prawa dla pionowego skierowania drzwi.

### Krótki opis metody `Apartment.simulate()`

Metoda ta jest sercem całego projektu, dlatego zawieramy krótki rozdział jej poświęcony. Działa ona osobno dla każdego rodzaju obiektów w kolejności: `Room`, `Radiator`, `Window`, `Door`. Metoda ta w kolejnych iteracjach `k in range(self.n_timeslips)` wylicza kolejne wartości macietzy `self.Matrix[k]` na podstawie wartości macierzy `self.Matrix[k-1]`. `k=0` jest rozpatrzony jako osobny przypadek.
Dla każdego rodzaju obiektów `Apartment.simulate()` zachowuje się zgodnie z opisami klas powyżej.

Żeby wymusić warunek von Neumanna dla ścian, na brzegach pokoi temperatura jest kopiowana z komórek obok.
Grzejniki po prostu dodają temperaturę zgodnie ze swoją funkcją `power`, okna przyjmują temperaturę na zewnątrz, a temperatura drzwi jest ujednolicana na każdym kroku.

# Wyniki eksperymentów

Poniżej przedstawimy opis oraz wyniki eksperymentów numerycznych zapowiedzianych we wstępie.

## *Czy grzejnik powinien znajdować się pod oknem?*

### Opis
W wielu polskich domach grzejniki znajdują się pod oknami. W tym eksperymencie sprawdzimy, czy takie ustawienie grzejników jest odpowiednie. 

Nasz eksperyment przeprowadzimy dla mieszkania składającego się z tylko jednego pokoju. W pokoju tym będzie dokładnie jeden grzejnik oraz dokładnie jedno okno. Rozważymy dwa warianty rozmieszczenia grzejnika: w jednym będzie się on znajdował bezpośrednio pod oknem, a w drugim będzie on bezpośrednio naprzeciwko okna. Eksperymenty numeryczne będą przeprowadzone dla $T = 24$ oraz $T_{out}(t) = 288.15 - 5\sin(\frac{12(t+2)}{\pi})$. Temperatura będzie się zmieniała w sposób sinusoidalny;

Zakładamy, że moc grzejnika jest stała i wynosi ona $120\frac{W}{m^2}$, co jest mocą na przykład dla [tego](https://cieplo-zimno.com/produkt/grzejnik-akumulacyjny-lucht-1800w/) grzejnika.

szkic pokoju

Dokładniejszy opis pokoi można znaleźć w kodzie źródłowym.

### Wyniki

wykresy

Jak wskazują wykresy, zdecydowanie bardziej opłaca się mieć grzejnik pod oknem aniżeli naprzeciwko okna.

## *Czy warto wyłączać grzejniki przed wyjściem z domu?*

### Opis
W tym eksperymencie sprawdzimy, czy bardziej opłaca się zostawić odkręcone pokrętła naszego grzejnika pod naszą nieobecność w domu czy też podgrzać dom na nowo po naszej nieobecności, na przykład po powrocie ze szkoły czy z pracy. W tym celu rozważymy mieszkanie opisane poniższym rysunkiem oraz trzy warianty temperatury na zewnątrz: zimnej (ok. $0℃$), bardzo zimnej ($-20℃$)  oraz bardzo bardzo zimnej ($-70℃$). Co prawda tak zimne temperatury rzadko mają miejsce w Polsce, ale ich symulacja lepiej pozwoli nam odpowiedzieć na pytanie postawione w zagadnieniu. W każdym z trzech wariantów temperatura będzie nieco się wahać, osiągając swoje minimum nad ranem oraz maksimum po południu.

# Podsumowanie
???